# HW04

In this dataset our desired target for classification task will be `converted` variable - has the client signed up to the platform or not.  

### Data preparation
- Check if the missing values are presented in the features.
- If there are missing values:
    - For caterogiral features, replace them with 'NA'
    - For numerical features, replace with with 0.0  
Split the data into 3 parts: train/validation/test with 60%/20%/20% distribution. Use `train_test_split` function for that with `random_state=1

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

In [4]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
)

In [ ]:
# Standardize column names (following notebook style)
df.columns = df.columns.str.lower().str.replace(" ", "_")

The target variable 'converted' is already in 0/1 format

In [6]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [10]:
# check dtypes
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [11]:
# Identify categorical and numerical features
numerical = [
    "number_of_courses_viewed",
    "annual_income",
    "interaction_count",
    "lead_score",
]
categorical = ["lead_source", "industry", "employment_status", "location"]

# Data Preparation: Handle missing values
for col in categorical:
    # For categorical features, replace missing values with 'NA'
    df[col] = df[col].fillna("NA")

for col in numerical:
    # For numerical features, replace missing values with 0.0
    df[col] = df[col].fillna(0.0)

### To split data into three parts => Train/Validation/Test

In [12]:
# check how many data
n = len(df)
n

1462

In [13]:
# Split the data into 3 parts: train/validation/test with 60%/20%/20% distribution.
# First split: full_train (80%) and test (20%)
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

# Second split: train (60%) and validation (20%) from full_train (80% * 0.25 = 20%)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [14]:
# Reset index
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_full_train = df_full_train.reset_index(drop=True)  # Used for Q5/Q6 CV

In [15]:
# Extract the target variables
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values
y_full_train = df_full_train.converted.values

In [16]:
# Delete the target variable from the feature dataframes
del df_train["converted"]
del df_val["converted"]
del df_test["converted"]
del df_full_train["converted"]

### Question 1: ROC AUC feature importance

ROC AUC could also be used to evaluate feature importance of numerical variables. 

Let's do that

* For each numerical variable, use it as score (aka prediction) and compute the AUC with the `y` variable as ground truth.
* Use the training dataset for that


If your AUC is < 0.5, invert this variable by putting "-" in front

(e.g. `-df_train['balance']`)

AUC can go below 0.5 if the variable is negatively correlated with the target variable. You can change the direction of the correlation by negating this variable - then negative correlation becomes positive.

Which numerical variable (among the following 4) has the highest AUC?

- `lead_score`
- `number_of_courses_viewed`
- `interaction_count`
- `annual_income`

In [17]:
# Question 1: ROC AUC feature importance
print("--- Question 1: ROC AUC Feature Importance ---")

scores = {}

for col in numerical:
    # Use the numerical feature as the score (prediction)
    auc = roc_auc_score(y_train, df_train[col])

    # If AUC < 0.5, invert the variable and recalculate AUC
    if auc < 0.5:
        auc = roc_auc_score(y_train, -df_train[col])
        print(f"Feature {col} was inverted.")

    scores[col] = auc
    print(f"{col:25} {auc:.4f}")

# Find the feature with the highest AUC
best_feature_q1 = max(scores, key=scores.get)

print(
    f"\nThe numerical variable with the highest AUC is: {best_feature_q1} ({scores[best_feature_q1]:.4f})"
)

--- Question 1: ROC AUC Feature Importance ---
number_of_courses_viewed  0.7636
annual_income             0.5520
interaction_count         0.7383
lead_score                0.6145

The numerical variable with the highest AUC is: number_of_courses_viewed (0.7636)
